### Amazon Comprehend Demo

***
Copyright [2017]-[2017] Amazon.com, Inc. or its affiliates. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. A copy of the License is located at

http://aws.amazon.com/apache2.0/

or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
***

### Prerequisites:

#### Identity and Acces Management

The user or role that executes the commands must have permissions in AWS Identity and Access Management (IAM) to perform those actions. AWS provides a set of managed policies that help you get started quickly. For our example, you should apply the following managed policy to your user or role:

    ComprehendReadOnly

Be aware that we recommend you follow AWS IAM best practices for production implementations, which is out of scope for this workshop.

In [ ]:
import boto3
import gzip
import json
from pprint import pprint

comprehend = boto3.client('comprehend', region_name='eu-west-1')

In [ ]:
# download review dataset
!curl -O http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz

In [ ]:
filename = 'reviews_Amazon_Instant_Video_5.json.gz'
f = gzip.open(filename, 'r') 
out = [] 
x = 50 # only process the first 50 entries 
for line in f: 
    x -= 1
    if x == 0:
        break
    review = json.loads(line)
    # get sentiment for reviewText
    reviewText = review['reviewText']
    if len(reviewText) > 5000: # only supporting up to 5000 Bytes, skipping entry
        print ('Skipping: %s' % reviewText)
    else:
        textSentiment = comprehend.detect_sentiment(
                            Text=reviewText,
                            LanguageCode='en'
                            )

        out.append([review['reviewText'],review['asin'],textSentiment['Sentiment'],textSentiment['SentimentScore']['Positive'],textSentiment['SentimentScore']['Negative'],textSentiment['SentimentScore']['Neutral'],textSentiment['SentimentScore']['Mixed']])   



In [ ]:
pprint (out)

In [ ]:
with open('sentiment.csv', 'w') as csvfile:
            linewriter = csv.writer(csvfile, delimiter=';',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            linewriter.writerow (['review','asin','Sentiment','Positive','Negative','Neutral','Mixed'])
            for all in out:
                linewriter.writerow(all)